In [1]:
import os, numpy as np, onnx, onnxruntime as ort
from pathlib import Path

print('onnxruntime', ort.__version__)

onnxruntime 1.19.2


In [7]:
# 🔧 ---------------- Configuration ----------------
MODEL_PATH = Path('./RealTimeAlignment/onnx_no-residual/submodule_onnx/submodule_embed.onnx')   # update as necessary
DUMP_DIR   = Path('embed_dumps')
DUMP_DIR.mkdir(exist_ok=True)

# Optional: external input file (.npy or .txt); leave None to use random data
INPUT_FILE = None
BATCH_SIZE = 1
# -------------------------------------------------

In [8]:
# Prepare input
def load_input():
    if INPUT_FILE is None:
        print('[INFO] No INPUT_FILE provided – generating random input')
        x = np.random.randn(BATCH_SIZE, 6).astype(np.float16)
    else:
        if INPUT_FILE.suffix == '.npy':
            x = np.load(INPUT_FILE).astype(np.float16)
        else:
            x = np.loadtxt(INPUT_FILE, dtype=np.float16)
        x = x.reshape(BATCH_SIZE, 6).astype(np.float16)
    np.savetxt(DUMP_DIR/'dense1_input.txt', x.astype(np.float32).flatten(), fmt='%.6f')
    return x

x_in = load_input()
print('Input shape:', x_in.shape)

[INFO] No INPUT_FILE provided – generating random input
Input shape: (1, 6)


In [9]:
# Dump weights/biases
model = onnx.load(str(MODEL_PATH))
initializer_map = {init.name: init for init in model.graph.initializer}

def tensor_to_numpy(tensor):
    return np.frombuffer(tensor.raw_data,
                         dtype=onnx.mapping.TENSOR_TYPE_TO_NP_TYPE[tensor.data_type]
                         ).reshape(tensor.dims)

def dump_tensor(name, arr):
    np.savetxt(DUMP_DIR/f'{name}.txt', arr.astype(np.float32).flatten(), fmt='%.6f')

for name, tensor in initializer_map.items():
    arr = tensor_to_numpy(tensor)
    dump_tensor(f'weights_{name}', arr)
    print(f'Dumped {name}  shape={arr.shape}')

Dumped 1.bias  shape=(128,)
Dumped 4.bias  shape=(128,)
Dumped onnx::MatMul_13  shape=(6, 128)
Dumped onnx::MatMul_14  shape=(128, 128)


/tmp/ipykernel_1210996/1062612749.py:7: DeprecationWarning: `mapping.TENSOR_TYPE_TO_NP_TYPE` is now deprecated and will be removed in a future release.To silence this warning, please use `helper.tensor_dtype_to_np_dtype` instead.
  dtype=onnx.mapping.TENSOR_TYPE_TO_NP_TYPE[tensor.data_type]


In [ ]:
# Inference with intermediate tensors
sess_opts = ort.SessionOptions()
sess_opts.graph_optimization_level = ort.GraphOptimizationLevel.ORT_DISABLE_ALL
session = ort.InferenceSession(str(MODEL_PATH), sess_opts, providers=['CPUExecutionProvider'])

wanted_ops = ('Gemm', 'LeakyRelu')
output_names = []
for node in model.graph.node:
    if node.op_type in wanted_ops:
        output_names.extend(node.output)
output_names.append(session.get_outputs()[0].name)
# remove duplicates while preserving order
seen = set()
unique_names = []
for n in output_names:
    if n not in seen:
        unique_names.append(n)
        seen.add(n)
input_name = session.get_inputs()[0].name
results = session.run(unique_names, {input_name: x_in})
dense1_out, relu1_out, dense2_out, relu2_out = results
dump_tensor('dense1_output', dense1_out)
dump_tensor('relu1_input', dense1_out)
dump_tensor('relu1_output', relu1_out)
dump_tensor('dense2_input', relu1_out)
dump_tensor('dense2_output', dense2_out)
dump_tensor('relu2_input', dense2_out)
dump_tensor('relu2_output', relu2_out)
dump_tensor('final_output', relu2_out)
print('Dumped intermediate and final outputs')